In [1]:
import numpy as np
import json
import sys
import pickle
import time
import cvxopt as cvx
# from cvxopt import matrix, solvers

In [2]:
def read_data (datafile="../dataset/SVM/train.csv", entryno=2):
    lines = np.array([(line.rstrip('\n')).split(',') for line in open(datafile)], dtype=np.int64)
    lines = lines[ (lines[:,-1] == entryno) | (lines[:,-1] == entryno+1) ]
    np.random.seed(0) # Deterministically random
    np.random.shuffle(lines)
#     print (lines[:10])
#     lines = lines[:200,:]
    X = lines[:,:-1] / 255.0
    Y = lines[:,-1] * 2 - 5 # for having y = {-1, 1}
    
#     print (X.shape, X)
#     np.set_printoptions(threshold=np.nan)
#     print (Y.shape, Y)
#     print (lines.shape, lines)
#     print (np.max(X[0]), np.min(X[0]))

    return X, Y
    
X, Y = read_data('../dataset/SVM/train.csv')
testX, testY = read_data('../dataset/SVM/test.csv')
print (X.shape, Y.shape, testX.shape, testY.shape)

(4000, 784) (4000,) (2042, 784) (2042,)


In [3]:
# LibSVM
from libsvm.python.svmutil import *

# prob = svm_problem(Y, X)

# param = svm_parameter()
# param.kernel_type = LINEAR
# param.C = 1

# m=svm_train(prob, param)

# correct = 0
# svm_predict(Y, X, m)[0]
# result = testY == svm_predict(testY, testX, m)[0]
# np.sum(result)

In [4]:
# Finding the SVM alphas
X = np.matrix(X)
Y = np.matrix(Y).T

m, n = X.shape
print (X.shape, Y.shape, m, n)

(4000, 784) (4000, 1) 4000 784


In [5]:
# def gaussianB (X):
#     X = np.matrix(X)
#     m, n = X.shape
#     XtX = np.multiply(X, X)
#     XtX = np.sum(XtX, axis=1).reshape(m,1)
#     XtX = np.repeat(XtX, m, axis=1)
# #     print (XtX)
#     print ((X*X.T).shape)
#     return XtX + XtX.T - 2*X*X.T

# def gaussianM (X):
#     XB = gaussianB(X)
#     gamma = 0.05
#     return np.exp ( -gamma * XB)
    
# M = gaussianM(X)
        
# print (M)

In [6]:
def gaussianB (X):
    X = np.matrix(X)
    m, n = X.shape
    XXt = X * X.T
    D = np.diag(XXt).reshape(m, 1)
    D1 = np.matrix(np.repeat(D, m, axis=1))
    D2 = D1.T
#     print (XXt, D, D1, D2)
    return D1 + D2 - 2 * XXt
    
    
    
#     m, n = X.shape
#     XtX = np.multiply(X, X)
#     XtX = np.sum(XtX, axis=1).reshape(m,1)
#     XtX = np.repeat(XtX, m, axis=1)
# #     print (XtX)
#     print ((X*X.T).shape)
#     return XtX + XtX.T - 2*X*X.T

def gaussianM (X):
    XB = gaussianB(X)
    gamma = 0.05
    return np.exp ( -gamma * XB)
    
M = gaussianM(X)
# HI = [[1,2,3,4],[4,5,6,7],[6,8,8,10]]   
# gaussianM (X)
# print (M)

In [7]:
mQ = Y * Y.T
print (mQ, mQ.shape)
# mQ = np.dot (mQ, M)
# print (mQ, mQ.shape)
mQ = np.multiply (mQ, M)
print (mQ, mQ.shape)
# for i in range (0, m):
#     for j in range (0, m):
#         Q[i][j] = 

[[ 1  1  1 ... -1  1 -1]
 [ 1  1  1 ... -1  1 -1]
 [ 1  1  1 ... -1  1 -1]
 ...
 [-1 -1 -1 ...  1 -1  1]
 [ 1  1  1 ... -1  1 -1]
 [-1 -1 -1 ...  1 -1  1]] (4000, 4000)
[[ 1.          0.00535561  0.00327795 ... -0.00578413  0.02642193
  -0.00182654]
 [ 0.00535561  1.          0.00262373 ... -0.00652064  0.04797889
  -0.00650381]
 [ 0.00327795  0.00262373  1.         ... -0.00788445  0.00323678
  -0.01077801]
 ...
 [-0.00578413 -0.00652064 -0.00788445 ...  1.         -0.01322943
   0.08776532]
 [ 0.02642193  0.04797889  0.00323678 ... -0.01322943  1.
  -0.00994946]
 [-0.00182654 -0.00650381 -0.01077801 ...  0.08776532 -0.00994946
   1.        ]] (4000, 4000)


In [8]:
Q = mQ

p = np.matrix(-np.ones((m, 1)))

G = np.matrix(np.vstack( (-np.identity(m), np.identity(m)) ))
h = np.matrix(np.vstack( (np.zeros((m,1)), np.ones((m,1))) ))

A = Y.T
b = 0

print (Q.shape, p.shape, G.shape, h.shape, A.shape, b)
# print (Q, p, G, h, A, b)

(4000, 4000) (4000, 1) (8000, 4000) (8000, 1) (1, 4000) 0


In [9]:
Q = cvx.matrix(Q)
p = cvx.matrix(p)
G = cvx.matrix(G)
h = cvx.matrix(h)
A = cvx.matrix(A, (1, m), 'd')
print (A, np.array(A).shape)
b = cvx.matrix(b, (1,1), 'd')

sol = cvx.solvers.qp(P=Q, q=p, G=G, h=h, A=A, b=b)

[-1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00  1.00e+00 -1.00e+00 ... ]
 (1, 4000)
     pcost       dcost       gap    pres   dres
 0: -2.5038e+02 -6.1151e+03  2e+04  2e+00  2e-15
 1: -1.6695e+02 -3.1888e+03  4e+03  2e-01  1e-15
 2: -2.1251e+02 -6.4884e+02  5e+02  1e-02  3e-15
 3: -2.6732e+02 -3.6833e+02  1e+02  2e-03  2e-15
 4: -2.8330e+02 -3.0848e+02  3e+01  3e-04  1e-15
 5: -2.8872e+02 -2.9361e+02  5e+00  4e-05  1e-15
 6: -2.9011e+02 -2.9076e+02  7e-01  4e-06  1e-15
 7: -2.9034e+02 -2.9037e+02  4e-02  1e-07  1e-15
 8: -2.9035e+02 -2.9035e+02  1e-03  3e-09  1e-15
 9: -2.9035e+02 -2.9035e+02  3e-05  3e-11  1e-15
Optimal solution found.


In [10]:
alphas = np.matrix(sol['x'])
print (type(alphas), alphas.shape, Y.shape)
# print (alphas)

<class 'numpy.matrixlib.defmatrix.matrix'> (4000, 1) (4000, 1)


In [11]:
testX = np.matrix(testX)
testY = np.matrix(testY)
print (X.shape, Y.shape, testX.shape, testY.shape)

(4000, 784) (4000, 1) (2042, 784) (1, 2042)


In [12]:
p, n = testX.shape

AlphaY = np.multiply (alphas, Y)
print (AlphaY.shape)
AlphaY = np.repeat(AlphaY, p, axis=1)
print (AlphaY.shape)

(4000, 1)
(4000, 2042)


In [13]:
def gaussianB (X, P):
    X = np.matrix(X)
    P = np.matrix(P)
    m, n = X.shape
    p, n = P.shape
    print (X.shape, P.shape)
    
    XPt = X * P.T
    D1 = np.diag(X * X.T).reshape(m, 1)
    D1 = np.matrix(np.repeat(D1, p, axis=1))
    D2 = np.diag(P * P.T).reshape(p, 1)
    D2 = np.matrix(np.repeat(D2, m, axis=1))
    D2 = D2.T
    
    print (D1.shape, D2.shape, XPt.shape)
    return D1 + D2 - 2 * XPt
    
    XXt = X * X.T
    D = np.diag(XXt).reshape(m, 1)
    D1 = np.matrix(np.repeat(D, m, axis=1))
    D2 = D1.T
#     print (XXt, D, D1, D2)
    return D1 + D2 - 2 * XXt
    

def gaussianM (X, P):
    XB = gaussianB(X, P)
    gamma = 0.05
    return np.exp ( -gamma * XB)

KM = gaussianM (X, testX)

(4000, 784) (2042, 784)
(4000, 2042) (4000, 2042) (4000, 2042)


In [14]:
# epsilon = 1e-5
# for idx, alp in enumerate(alphas):
#     if (alp - 0 > epsilon and 1 - alp > epsilon):
#         print (alp, idx)
#         testX = testX[idx]
#         KM = gaussianM (X, testX)
#         AlphaY = np.multiply (alphas, Y)
#         print (AlphaY.shape)
#         AlphaY = np.repeat(AlphaY, 1, axis=1)
#         print (AlphaY.shape)
#         KMalphaY = np.multiply (KM, AlphaY)
#         KMalphaY = np.sum(KMalphaY, axis=0)
#         print (KMalphaY.shape, KMalphaY)

#         correct = np.multiply(KMalphaY, testY) > 0
        
#         print (testX.shape)
# #         print (testY[0, idx], KMalphaY)
# #         break
#         b = float (testY[0, idx] - KMalphaY)
#         break
#         y = float (Y[idx])
        
#         w = np.matrix(W)
#         x = np.matrix(X[idx, :]).T
#         b = float (y - w*x)
#         print (y, w.shape, x.shape, b)
#         break
# print (b)

In [15]:
b = 1.4318501661556025
print (KM.shape)
KMalphaY = np.multiply (KM, AlphaY)
print (KMalphaY.shape)
KMalphaY = np.sum(KMalphaY, axis=0) + b
print (KMalphaY.shape, testY.shape, KMalphaY, testY)

correct = np.multiply(KMalphaY, testY) > 0
print (correct)
print (np.sum(correct))
acc = float(np.sum(correct, axis=1)) / len(testY.T)
print ("Accuracy: ", acc, len(testY))

(4000, 2042)
(4000, 2042)
(1, 2042) (1, 2042) [[2.74925648 2.98850562 2.38238369 ... 3.20057056 3.44911458 0.39226665]] [[ 1  1  1 ...  1  1 -1]]
[[ True  True  True ...  True  True False]]
1014
Accuracy:  0.49657198824681686 1


In [16]:
# def getWtX (x):
# #     print (alphas.shape, Y.shape, X.shape, x.shape)
#     WtX = X - x
#     WtX = np.multiply(WtX, WtX)
#     WtX = np.sum(WtX, axis=1)
# #     print (WtX.shape)

#     WtX = np.multiply(WtX, Y)
#     WtX = np.multiply(WtX, alphas)
#     WtX = np.sum(WtX, axis=0)
    
#     return float(WtX)

In [17]:
# epsilon = 1e-5
# print (alphas)
# for idx, alp in enumerate(alphas):
#     alp = float(alp)
#     if (alp - 0 > epsilon and 1 - alp > epsilon):
#         y = float (Y[idx])
#         x = np.matrix(X[idx, :])
# #         print (X, x, X -x)
#         print (X.shape, x.shape, (X-x).shape)
#         b = 1 - getWtX (x)
#         break
#         print (alp)
#         w = np.matrix(W)
#         x = np.matrix(X[idx, :]).T
#         b = float (y - w*x)
#         print (y, w.shape, x.shape, b)
#         break
# print (b)

In [18]:
correct = 0
for y, x in zip (Y, X):
    pred = getWtX (x) + b
    if (y * pred > 0):
        correct += 1
print ("Accuracy: ", correct / len(Y))

NameError: name 'getWtX' is not defined

In [ ]:
# alphas = alphas[:,0]
# print (alphas)

In [ ]:
epsilon = 1e-5
sv = []
for idx, alp in enumerate(alphas):
    if (alp - 0 > epsilon and 1 - alp > epsilon):
#         print (alp, idx)
        sv.append(alp)
print (len(sv))

In [ ]:
print ()

In [ ]:
A = np.matrix ([[1,2,3],[4,5,6]])
B = np.matrix ([[2],[3]])
print (A, B)
C = np.multiply(A, B)
C = np.multiply(C, B)
print (C)

In [ ]:
print (Y.shape, X.shape, alphas.shape)

In [ ]:
W = np.multiply(X, Y)
W = np.multiply(W, alphas)
print (W.shape, W)

In [ ]:
W = W.sum(axis=0)
print (W.shape, W)

In [ ]:
W = np.array(W)[0, :]
print (W.shape, W)

In [ ]:
print (np.sum(W))

In [ ]:
epsilon = 1e-5
for idx, alp in enumerate(alphas):
    if (alp - 0 > epsilon and 1 - alp > epsilon):
        print (alp)
        y = float (Y[idx])
        w = np.matrix(W)
        x = np.matrix(X[idx, :]).T
        b = float (y - w*x)
        print (y, w.shape, x.shape, b)
        break
print (b)

In [ ]:
W = np.matrix(W).T
print (W.shape, b, X.shape)

In [ ]:
pred = X*W
pred = np.array(pred)[:,0]
print (pred.shape, pred)

In [ ]:
values = np.sign(pred)
print (values)

In [ ]:
# Y = np.array(Y)[:,0]
print (Y)
accuracy = np.sum(values == Y) / len(Y)
print (accuracy)